<h1 style="text-align: center;">Lab session 1: 1D Continous Wavelet Transform</h1>
<h3 style="text-align: center;">Alexis Sonolet</h3>


## Prerequisite

### Install Anaconda and PyWavelet

* The simplest is to download the Anaconda distribution (including PyWavelet) : https://www.anaconda.com/ 
* Or follows these instructions: https://pywavelets.readthedocs.io/en/latest/install.html
* Read the documentation on Pywavelet: https://pywavelets.readthedocs.io/_/downloads/en/v0.5.1/pdf/
* Code available on Github (to know how it is implemented): https://github.com/PyWavelets/pywt/tree/580d79d9440ec0f4f936892e39c79ad13a8fd33b 

### Instructions

* Fill empty codes and answer the questions in the notebook
* Export into PDF, join the code (.ipynb) and send the results by email to: kevin.polisano@univ-grenoble-alpes.fr

### Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import pywt
from pylab import*
import scaleogram as scg
import itertools
import random

## Fourier transform limitation in a nutschell

**Question 1**: Let's compute the Fourier transform of 2 signals made of
1. The superposition of 4 sinusoids with different frequencies on $[0,1]$ (notes played together)
2. The concatenation of these 4 sinusoids each with duration $1/4$ on $[0,1]$ (notes played successively)

**Answer 1**:

In [ ]:
def fft_compute(y, T, N):
    f = np.linspace(0.0, 1.0/(2.0*T), N//2)
    fft_y = fft(y)
    fft_y = 2.0/N * np.abs(fft_y[0:N//2])
    return f, fft_y

N = 1000
T = 1.0/N
 
x = np.linspace(0, 1, num=N)
 
freq = [8, 20, 35, 50]
random.shuffle(freq)

y_superp = [sum([np.sin(2*np.pi*frequency*t) for frequency in freq]) for t in x]
y_concat = list(itertools.chain(*[[np.sin(2*np.pi*frequency*t) for t in time_range] for frequency, time_range in zip(freq, np.split(np.array(x), 4))]))
 
f1, fft_superp = fft_compute(y_superp, T, N)
f2, fft_concat = fft_compute(y_concat, T, N)

plt.figure(figsize=(12, 6))
plt.subplot(221)
plt.plot(x, y_superp)
plt.title('signal from superposition')
plt.xlabel('time')

plt.subplot(223)
plt.plot(x, y_concat)
plt.title('signal from concatenation')
plt.xlabel('time')

plt.subplot(222)
plt.plot(f1, fft_superp, 'r')
plt.xlim(0, 60)
plt.title('FFT of the superposition signal')
plt.xlabel('frequency')

plt.subplot(224)
plt.plot(f2, fft_concat, 'r')
plt.xlim(0, 60)
plt.title('FFT of the concatenation signal')
plt.xlabel('frequency')
 
plt.tight_layout()
plt.show()

**Question 2**: 
* Explain why the peaks on the concatenate version is more spread

The peaks are more spread on the concatenate version because we don't really have only four frequencies like in the superposition version : as a matter of facts as this is a concatenation and not a superposition, the final signal act like if he had way more frequencies and therefore the signal spreads more.

* Try to exchange the order of the concatenation. Conclusion?

Changing the order of the concatenation doesn't change it's fourier transform. It means that the signal is highly dependable on the four innitial frequencies.

## Some signals to play with

The *PyWavelet package* provides a list of signals demo:

In [ ]:
signals = pywt.data.demo_signal('list')
print(signals)

To get the values of a signal and display it:

In [ ]:
signal_length = 1024
y = pywt.data.demo_signal('Piece-Regular', signal_length)
plt.figure(figsize=(12, 6))
plt.plot(y)
plt.show()

**Question 3**: Display in a loop the others signal in different subplots. You will especially make appear both the real and imaginary parts of the Gabor signal.

In [ ]:
signal_length = 1024
plt.figure(figsize=(18, 20))

for n_subplot, signal in enumerate(pywt.data.demo_signal('list')):
    ax = plt.subplot2grid((5,4), (n_subplot//4, n_subplot%4))
    if signal in ['Gabor', 'sineoneoverx']:
        y = pywt.data.demo_signal(signal, None)
        y_real = list(map(np.real, y))
        y_imag = list(map(np.imag, y))
        real_plot = ax.plot(y_real)
        imag_plot = ax.plot(y_imag)
        plt.legend(("Real", "Imag"))
    else:
        y = pywt.data.demo_signal(signal, signal_length)
        ax.plot(y)
    plt.title(signal)
plt.show()

## Familiarize yourself with the family of continuous wavelets

Display the family of all wavelets available in *PyWavelet*:

In [ ]:
print(pywt.families(short=False))

For such a family, all the available wavelets are given by:

In [ ]:
print(pywt.wavelist('db')) # Daubechies wavelets

The **continuous wavelet** are given by:

In [ ]:
print(pywt.wavelist(kind='continuous'))

To read the characteristics of a particular wavelet:

In [ ]:
w = pywt.ContinuousWavelet('gaus2')
print(w)

### Vizualize some classical continuous wavelets

**Question 4**: Display in a loop all the continous wavelets in different subplots.

*Hint: To compute an approximation of the scaling function (psi) use the command `wavefun` (see documentation for more details: https://pywavelets.readthedocs.io/en/latest/ref/wavelets.html)*

In [ ]:
continuous_wavelets = pywt.wavelist(kind='continuous')
# 21 wavelets
signal_length = 1024
plt.figure(figsize=(16, 22))

for n_subplot, signal in enumerate(continuous_wavelets):
    wavelet = pywt.ContinuousWavelet(signal)
    psi, x = wavelet.wavefun(level=5)

    ax = plt.subplot2grid((6,4), (n_subplot//4, n_subplot%4))

    ax.plot(x, psi.real)
    plt.title(signal)
plt.show()

### Vizualize the scaling of the Morlet wavelet in time and frequency domains

**Question 5**: Plot wavelets at different scales from a given mother wavelet, as well as their Fourier transforms

To vizualize the mother wavelet one can use the `scaleogram` package (see the documentation https://github.com/alsauve/scaleogram/blob/master/doc/tests.ipynb and https://github.com/alsauve/scaleogram/blob/master/doc/scale-to-frequency.ipynb):

In [ ]:
axes = scg.plot_wav('morl', figsize=(14,3))
show()

wavelet = pywt.ContinuousWavelet('morl')
psi, time = wavelet.wavefun(level=8)
filles = [1/2, 1, 4]

plt.figure(figsize=(14, 4*3))
for index, scale in enumerate(filles):
    # fonction de base
    plt.subplot(int("32" + str(2*index + 1)))
    plt.title("scale : {}".format(scale))
    psi_fille = 1/np.sqrt(scale)*np.interp(time/scale, time, psi)
    plt.plot(time, psi_fille)
    
    # transformée de fourier
    plt.subplot(int("32" + str(2*index + 2)))
    plt.title("fourier transform for scale = {}".format(scale))
    fft_fille = np.abs(np.fft.fft(psi_fille))
    frequency = pywt.scale2frequency(wavelet, scale)
    freq = np.arange(len(time)) * 1/ (time[-1] - time[0])
    plt.plot(freq, fft_fille)
    plt.xlim(0, time[-1])
    plt.plot([frequency, frequency], [0, max(fft_fille)])


## The 1D Continuous Wavelet Transform (CWT)

### Compute the CWT

The function `pywt.cwt` enables to compute the CWT of a discrete signals at different scales.

See the documentation: https://pywavelets.readthedocs.io/en/latest/ref/cwt.html

**Question 6**: Apply the CWT to reproduce the 6 examples given in the course:

    * Example 1: pure cosine
    * Example 2: a Dirac
    * Example 3: the periodic square wave
    * Example 4: a modulated wave
    * Example 5: 2 sinusoids with noise
    * Example 6: Holder function of exponant 1/2

*Hint:* Consider for the first example a cosine with frequency $f=50~Hz$ on $[0,1]$ with a sampling rate $f_s=\frac{1}{400}$, compute its CWT onto scales from 1 to 50 and display the corresponding scalogram (by plotting the coef/freqs in output of `pywt.cwt`). Check that you vizually detect the frequency $f$.

In [ ]:
def plotting(x, signal, wavelet, scales, name, ymax=100):
    coefs, freqs = pywt.cwt(signal, scales, wavelet, sampling_period = (max(x)-min(x))/len(x))
    plt.figure(figsize=(14, 3))
    ax1 = plt.subplot(121)
    plt.title(name)
    plt.plot(x, signal)
    ax2 = plt.subplot(122)
    plt.title(name + " scaleogram ({} wavelet)".format(wavelet))
    plt.xlabel('time (s)')
    plt.ylabel('frequency (Hz)')
    plt.ylim(ymax=ymax)
    plt.contourf(x, freqs, abs(coefs), 100, cmap='jet')
    return ax1, ax2


In [ ]:
print("Example 1 : pure cosine")

freq = 50
samplingRate = 1/400
X = np.arange(0, 1, samplingRate)
pureCosine = np.cos(2 * freq * pi * X)
# Morlet wavelet
wavelet = 'morl'
scales = np.arange(1, 50)
plotting(X, pureCosine, wavelet, scales, 'pureCosine', 150)

In [ ]:
print("Example 2 : a Dirac")

samplingRate = 1/400
X = np.arange(0, 1, samplingRate)
dirac = np.zeros(len(X))
dirac[int(len(X)/2)] = 1
scales = np.arange(1, 70)
wavelet = 'morl'
plotting(X, dirac, wavelet, scales, 'Dirac', 325)

In [ ]:
print("Example 3 : the periodic square wave")

samplingRate = 1/400
X = np.arange(0, 1, samplingRate)
square = np.zeros(len(X))
for i in range(len(X)):
    if (i//50)%2 == 0:
        square[i] = 1
scales = np.arange(1, 50)
wavelet = 'morl'
plotting(X, square, wavelet, scales, 'Periodic square wave', 200)

In [ ]:
print("Example 4: a modulated wave")

samplingRate = 1/400
X = np.arange(0, 1, samplingRate)
modulated = np.cos(2*50*pi*X**2)
scales = np.arange(1, 100)
wavelet = 'morl'
plotting(X, modulated, wavelet, scales, 'Modulated wave', 200)

In [ ]:
print("Example 5: 2 sinusoids with noise")

samplingRate = 1/400
X = np.arange(0, 1, samplingRate)
noiseSin = np.sin(2*20*pi*X) + np.sin(2*80*pi*X) + np.random.normal(0, 0.3, len(X))
scales = np.arange(1, 50)
wavelet = 'morl'
plotting(X, noiseSin, wavelet, scales, 'Sinusoid with noise', 150)

In [ ]:
print("Example 6: Holder function of exponant 1/2")

samplingRate = 1/400
X = np.arange(0, 1, samplingRate)
holder = np.sqrt(np.abs(np.cos(2*pi*X)))
scales = np.arange(1, 60)
wavelet = 'morl'
plotting(X, holder, wavelet, scales, 'square Holder function', 100)

**Question 7**: Same question applied to both examples of Question 1 (superposition and concatenation). Can you detect all the frequencies and their localization?

In [ ]:
N = 1000
T = 1.0/N

x = np.linspace(0, 1, num=N)

freq = [8, 20, 35, 50]

y_superp = [sum([np.sin(2*np.pi*frequency*t) for frequency in freq]) for t in x]
y_concat = list(itertools.chain(*[[np.sin(2*np.pi*frequency*t) for t in time_range] for frequency, time_range in zip(freq, np.split(np.array(x), 4))]))

plt.figure(figsize=(14, 2*4))
samplingRate = 1/400
scales = np.arange(1, 200)
wavelet = 'morl'
plotting(x, y_superp, wavelet, scales, 'superposition', 100)
plotting(x, y_concat, wavelet, scales, 'concaténation', 100)

- On the superposition graph, we can see 4 horizontal domains :
    - the first one at around 10Hz
    - the second one at 20Hz
    - the third one at 35Hz
    - the last one at around 55Hz

  These domains match perfectly the four frequencies we had (8, 20, 35 and 50Hz)
  </br>
  
- On the concatenation graph we can also see 4 groups :
    - the first one at 20Hz
    - the second one at around 40Hz
    - the third one at 50Hz
    - the last one at around 10Hz
    
  Once again we managed to find the four base frequencies of the signal. </br>
  But unlike the Fourier transform we can see with this graph when does the change of frequency occurs.

**Question 8**: After applying the CWT to the previous demo-signal 'Piece-Regular', you will emulate the maxima lines by finding local peaks at each scale (without chain them)

Suggestions:
* https://github.com/MonsieurV/py-findpeaks
* https://github.com/buckie/wtmm-python

In [ ]:
from scipy.signal import find_peaks

scales = np.arange(1,100)
signal_length = 1024
X = np.linspace(0,1,signal_length)
pieceRegular = pywt.data.demo_signal('Piece-Regular', signal_length)
wavelet ='morl'
ax1, ax2 = plotting(X, pieceRegular, wavelet, scales, "Piece-Regular", 150)

peaks, _ = find_peaks(pieceRegular)

for p in peaks:
    ax1.plot([X[p],X[p]],[-15,50],'r')

plt.show()


**Question 9**: Compute the CWT of a fractional Brownian motion with Hölder exponent 1/2

Use the `fbm` package: https://github.com/crflynn/fbm 

**Question 10 (Bonus)**: Recover the Hölder exponent by computing the slope of decrease of the wavelet coefficients.

In [ ]:
from fbm import FBM

print("Question 9")

f = FBM(1024, 0.5)
t_values = f.times()
fbm_sample = np.array(f.fbm())

scales = np.arange(1, 100)
wavelet = 'morl'

plotting(t_values, fbm_sample, wavelet, scales, "FBM", 80)